In [ ]:
API_KEY = "DEszypg8ixfdrtYL1ad9cnTi1qJPJKvso6GEtUIk"

import requests
from datetime import datetime

url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-07&api_key={API_KEY}"
asteroids_data = []
target = 200
while len(asteroids_data)<target:
  response = requests.get(url)
  data = response.json()
  details = data['near_earth_objects']

  for date,asteroid_details in details.items():
    for ast in asteroid_details:
      for approach in ast['close_approach_data']:
            astronomical = approach['miss_distance']['astronomical']
            missDistanceKm = approach['miss_distance']['kilometers']
            missDistanceLun = approach['miss_distance']['miles']
            orbitingBody = approach['orbiting_body']
      asteroids_data.append(dict(
       id = int(ast['id']),
       noReferenceId = int(ast['neo_reference_id']),
       name = ast['name'],
       magnitude = ast['absolute_magnitude_h'],
       absolutemagnitudeH = ast['absolute_magnitude_h'],
       estimatedDiameterMax = ast['estimated_diameter']['kilometers']['estimated_diameter_max'],
        estimatedDiameterMin = ast['estimated_diameter']['kilometers']['estimated_diameter_min'],
       isPotentiallyHazardousAsteroid = ast['is_potentially_hazardous_asteroid'],
       closeApproachDate = datetime.strptime(ast['close_approach_data'][0]['close_approach_date'], "%Y-%m-%d"),
       relativeVelocity = float(ast['close_approach_data'][0]['relative_velocity']['kilometers_per_second']),
        astronomicalMissDistance = float(astronomical),
        misDistanceKilometre = float(missDistanceKm),
        misDistanceMiles = float(missDistanceLun),
        orbitingBody = orbitingBody
      ))
      if len(asteroids_data) >= target:
        break
    if len(asteroids_data)>=target:
      break
  url = data['links']['next']

In [ ]:
asteroids_data

[{'id': 2415949,
  'noReferenceId': 2415949,
  'name': '415949 (2001 XY10)',
  'magnitude': 19.37,
  'absolutemagnitudeH': 19.37,
  'estimatedDiameterMax': 0.7944013596,
  'estimatedDiameterMin': 0.3552670883,
  'isPotentiallyHazardousAsteroid': False,
  'closeApproachDate': datetime.datetime(2024, 1, 2, 0, 0),
  'relativeVelocity': 15.8905264223,
  'astronomicalMissDistance': 0.3372535274,
  'misDistanceKilometre': 50452409.349026635,
  'misDistanceMiles': 31349673.465810083,
  'orbitingBody': 'Earth'},
 {'id': 3160747,
  'noReferenceId': 3160747,
  'name': '(2003 SR84)',
  'magnitude': 26.0,
  'absolutemagnitudeH': 26.0,
  'estimatedDiameterMax': 0.0375007522,
  'estimatedDiameterMin': 0.0167708462,
  'isPotentiallyHazardousAsteroid': False,
  'closeApproachDate': datetime.datetime(2024, 1, 2, 0, 0),
  'relativeVelocity': 10.7191818981,
  'astronomicalMissDistance': 0.1323425924,
  'misDistanceKilometre': 19798169.933318187,
  'misDistanceMiles': 12302012.344671475,
  'orbitingBody':

In [ ]:
data

{'links': {'next': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-13&end_date=2024-01-19&detailed=false&api_key=DEszypg8ixfdrtYL1ad9cnTi1qJPJKvso6GEtUIk',
  'previous': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-07&detailed=false&api_key=DEszypg8ixfdrtYL1ad9cnTi1qJPJKvso6GEtUIk',
  'self': 'http://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-07&end_date=2024-01-13&detailed=false&api_key=DEszypg8ixfdrtYL1ad9cnTi1qJPJKvso6GEtUIk'},
 'element_count': 131,
 'near_earth_objects': {'2024-01-13': [{'links': {'self': 'http://api.nasa.gov/neo/rest/v1/neo/3147312?api_key=DEszypg8ixfdrtYL1ad9cnTi1qJPJKvso6GEtUIk'},
    'id': '3147312',
    'neo_reference_id': '3147312',
    'name': '(2003 BN1)',
    'nasa_jpl_url': 'https://ssd.jpl.nasa.gov/tools/sbdb_lookup.html#/?sstr=3147312',
    'absolute_magnitude_h': 20.59,
    'estimated_diameter': {'kilometers': {'estimated_diameter_min': 0.2025606009,
      'estimated_diameter_max': 0.4529392731},
     'me

In [ ]:
import sqlite3

conn = sqlite3.connect('asteroids.db')
cursor = conn.cursor()

In [ ]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS asteroids (
    id INTEGER,
    name VARCHAR,
    absolutemagnitudeH FLOAT,
    estimatedDiameterMin FLOAT,
    estimatedDiameterMax FLOAT,
    isPotentiallyHazardousAsteroid BOOLEAN
)
""")

In [ ]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS close_approach (
    noReferenceId INTEGER,
    closeApproachDate TEXT,
    relativeVelocity FLOAT,
    astronomicalMissDistance FLOAT,
    misDistanceKilometre FLOAT,
    misDistanceMiles FLOAT,
    orbitingBody VARCHAR
)
""")

In [ ]:
for asteroid in asteroids_data:
    cursor.execute("""
        INSERT OR IGNORE INTO asteroids (
            id, name, absolutemagnitudeH,
            estimatedDiameterMax, estimatedDiameterMin, isPotentiallyHazardousAsteroid
        ) VALUES (?, ?, ?, ?, ?, ?)
    """, (
        asteroid['id'],
        asteroid['magnitude'],
        asteroid['absolutemagnitudeH'],
        asteroid['estimatedDiameterMax'],
        asteroid['estimatedDiameterMin'],
        asteroid['isPotentiallyHazardousAsteroid']
    ))
conn.commit()


In [ ]:
for asteroid in asteroids_data:
  cursor.execute("""
        INSERT INTO close_approach (
            noReferenceId, closeApproachDate, relativeVelocity,
            astronomicalMissDistance, misDistanceKilometre, misDistanceMiles, orbitingBody
        ) VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (
        asteroid['id'],
        asteroid['closeApproachDate'].strftime("%Y-%m-%d"),
        float(asteroid['relativeVelocity']),
        float(asteroid['astronomicalMissDistance']),
        float(asteroid['misDistanceKilometre']),
        float(asteroid['misDistanceMiles']),
        asteroid['orbitingBody']
    ))
conn.commit()


In [ ]:
cursor.execute("""SELECT noReferenceId, AVG(relativeVelocity) AS average_velocity FROM close_approach WHERE orbitingBody = 'Earth' GROUP BY noReferenceId ORDER BY average_velocity DESC""")

result = cursor.fetchall()

import pandas as pd
df = pd.DataFrame(result)
df.columns = [x[0] for x in cursor.description]
df

,noReferenceId,average_velocity
0,3790060,43.966629
1,3645001,37.852109
2,2434188,33.837342
3,3147312,32.092148
4,3596491,29.585502
...,...,...
181,3751529,3.725730
182,2676480,3.725730
183,3836297,1.608150
184,54416849,1.437722


In [ ]:
!pip install streamlit streamlit_option_menu  # installing streamlit and streamlit_option_menu packages

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.3/829.3 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.8 MB/s eta 0:00:00


In [ ]:
%%writefile app.py

import streamlit as st
import sqlite3
import pandas as pd
from streamlit_option_menu import option_menu
from datetime import datetime

st.set_page_config(layout='wide')

st.title('ASTEROID DATA ANALYSIS DASHBOARD')

# Sidebar Navigation
with st.sidebar:
    select = option_menu('Main Menu', ['Home', 'Filter Criteria', 'Upload & Modifying', 'Delete'])

# Connect to SQLite DB
conn = sqlite3.connect('asteroids.db', check_same_thread=False)

# ========== FILTER CRITERIA PAGE ==========
if select == "Filter Criteria":
    st.subheader("🔍 Filter Asteroids Based on Criteria")
    col1, col2, col3 = st.columns(3)

    with col1:
        mag = st.slider("Magnitude H", 13.0, 34.0, (13.0, 34.0))
        diameter = st.slider("Estimated Diameter (km)", 0.0, 10.0, (0.0, 10.0))

    with col2:
        velocity = st.slider("Velocity (km/h)", 0.0, 140000.0, (0.0, 140000.0))
        hazardous = st.selectbox("Hazardous?", ["All", "Yes", "No"])

    with col3:
        date_range = st.date_input("Approach Date Range", [datetime(2024, 1, 1), datetime(2025, 12, 31)])

    if st.button("Apply Filter"):
        query = """
        SELECT a.name, a.absoluteMagnitudeH, a.estimatedDiameterMin, a.estimatedDiameterMax,
               a.isPotentiallyHazardousAsteroid, ca.closeApproachDate, ca.relativeVelocity,
               ca.astronomicalMissDistance
        FROM asteroids a
        JOIN close_approach ca ON a.id = ca.noReferenceId
        WHERE a.absoluteMagnitudeH BETWEEN ? AND ?
          AND a.estimatedDiameterMax BETWEEN ? AND ?
          AND ca.relativeVelocity BETWEEN ? AND ?
          AND ca.closeApproachDate BETWEEN ? AND ?
        """
        params = [mag[0], mag[1], diameter[0], diameter[1],
                  velocity[0], velocity[1], date_range[0], date_range[1]]

        if hazardous == "Yes":
            query += " AND a.isPotentiallyHazardousAsteroid = 1"
        elif hazardous == "No":
            query += " AND a.isPotentiallyHazardousAsteroid = 0"

        df = pd.read_sql_query(query, conn, params=params)
        st.dataframe(df, use_container_width=True)

# Define your queries with display names
queries = {
    "Count how many times each asteroid has approached Earth":
        """
        SELECT ca.noReferenceId, a.name, COUNT(*) AS approach_count
        FROM close_approach ca
        JOIN asteroids a ON ca.noReferenceId = a.id
        WHERE ca.orbitingBody = 'Earth'
        GROUP BY ca.noReferenceId, a.name
        ORDER BY approach_count DESC;
        """,

    "Average velocity of each asteroid over multiple approaches":
        """
        SELECT ca.noReferenceId, a.name, AVG(ca.relativeVelocity) AS avg_velocity
        FROM close_approach ca
        JOIN asteroids a ON ca.noReferenceId = a.id
        GROUP BY ca.noReferenceId, a.name
        ORDER BY avg_velocity DESC;
        """,

    "List top 10 fastest asteroids":
        """
        SELECT a.name, MAX(ca.relativeVelocity) AS max_velocity
        FROM close_approach ca
        JOIN asteroids a ON ca.noReferenceId = a.id
        GROUP BY a.name
        ORDER BY max_velocity DESC
        LIMIT 10;
        """,

    "Find potentially hazardous asteroids that have approached Earth more than 3 times":
        """
        SELECT a.name, COUNT(*) AS approach_count
        FROM close_approach ca
        JOIN asteroids a ON ca.noReferenceId = a.id
        WHERE a.isPotentiallyHazardousAsteroid = 1 AND ca.orbitingBody = 'Earth'
        GROUP BY a.name
        HAVING approach_count > 3
        ORDER BY approach_count DESC;
        """,

    "Find the month with the most asteroid approaches":
        """
        SELECT strftime('%Y-%m', closeApproachDate) AS month, COUNT(*) AS approach_count
        FROM close_approach
        GROUP BY month
        ORDER BY approach_count DESC
        LIMIT 1;
        """,

    "Get the asteroid with the fastest ever approach speed":
        """
        SELECT a.name, ca.relativeVelocity
        FROM close_approach ca
        JOIN asteroids a ON ca.noReferenceId = a.id
        ORDER BY ca.relativeVelocity DESC
        LIMIT 1;
        """,

    "Sort asteroids by maximum estimated diameter (descending)":
        """
        SELECT name, estimatedDiameterMax
        FROM asteroids
        ORDER BY estimatedDiameterMax DESC;
        """,

    "An asteroid whose closest approach is getting nearer over time":
        """
        SELECT a.name, ca.closeApproachDate, ca.astronomicalMissDistance
        FROM close_approach ca
        JOIN asteroids a ON ca.noReferenceId = a.id
        WHERE ca.orbitingBody = 'Earth'
        ORDER BY a.name, ca.closeApproachDate ASC;
        """,

    "Display name, date and miss distance of closest approach to Earth":
        """
        SELECT a.name, ca.closeApproachDate, ca.astronomicalMissDistance
        FROM close_approach ca
        JOIN asteroids a ON ca.noReferenceId = a.id
        WHERE ca.orbitingBody = 'Earth'
        ORDER BY ca.astronomicalMissDistance ASC
        LIMIT 1;
        """,

    "List asteroids with velocity > 50,000 km/h":
        """
        SELECT a.name, ca.relativeVelocity
        FROM close_approach ca
        JOIN asteroids a ON ca.noReferenceId = a.id
        WHERE ca.relativeVelocity > 50000
        ORDER BY ca.relativeVelocity DESC;
        """,

    "Count how many approaches happened per month":
        """
        SELECT strftime('%Y-%m', closeApproachDate) AS month, COUNT(*) AS approach_count
        FROM close_approach
        GROUP BY month
        ORDER BY month;
        """,

    "Find asteroid with the highest brightness (lowest magnitude value)":
        """
        SELECT name, absolutemagnitudeH
        FROM asteroids
        ORDER BY absolutemagnitudeH ASC
        LIMIT 1;
        """,

    "Get number of hazardous vs non-hazardous asteroids":
        """
        SELECT isPotentiallyHazardousAsteroid AS hazardous, COUNT(*) AS total
        FROM asteroids
        GROUP BY hazardous;
        """,

    "Find asteroids that passed closer than the Moon (less than 1 LD)":
        """
        SELECT a.name, ca.closeApproachDate, ca.astronomicalMissDistance
        FROM close_approach ca
        JOIN asteroids a ON ca.noReferenceId = a.id
        WHERE ca.astronomicalMissDistance < 1
        ORDER BY ca.astronomicalMissDistance ASC;
        """,

    "Find asteroids that came within 0.05 AU":
        """
        SELECT a.name, ca.closeApproachDate, ca.astronomicalMissDistance
        FROM close_approach ca
        JOIN asteroids a ON ca.noReferenceId = a.id
        WHERE ca.astronomicalMissDistance < 0.05
        ORDER BY ca.astronomicalMissDistance ASC;
        """,

    "Find the longest gap (in days) between two consecutive approaches of the same asteroid":
        """
        SELECT name, MAX(gap_days) AS max_gap
        FROM (
            SELECT a.name,
                   julianday(ca2.closeApproachDate) - julianday(ca1.closeApproachDate) AS gap_days
            FROM close_approach ca1
            JOIN close_approach ca2 ON ca1.noReferenceId = ca2.noReferenceId
            JOIN asteroids a ON ca1.noReferenceId = a.id
            WHERE ca2.closeApproachDate > ca1.closeApproachDate
            GROUP BY ca1.noReferenceId, ca1.closeApproachDate
        ) sub
        GROUP BY name
        ORDER BY max_gap DESC;
        """,

    "List asteroids that have approached Earth every year over the last 5 years":
        """
        SELECT a.name
        FROM close_approach ca
        JOIN asteroids a ON ca.noReferenceId = a.id
        WHERE ca.orbitingBody = 'Earth'
          AND strftime('%Y', ca.closeApproachDate) >= strftime('%Y', 'now', '-4 years')
        GROUP BY a.name
        HAVING COUNT(DISTINCT strftime('%Y', ca.closeApproachDate)) = 5;
        """,

    "Top 5 asteroids with most frequent approaches in a single year":
        """
        SELECT a.name, strftime('%Y', ca.closeApproachDate) AS year, COUNT(*) AS approaches
        FROM close_approach ca
        JOIN asteroids a ON ca.noReferenceId = a.id
        GROUP BY a.name, year
        ORDER BY approaches DESC
        LIMIT 5;
        """,

    "Find earliest recorded approach to Earth by any asteroid":
        """
        SELECT a.name, ca.closeApproachDate
        FROM close_approach ca
        JOIN asteroids a ON ca.noReferenceId = a.id
        WHERE ca.orbitingBody = 'Earth'
        ORDER BY ca.closeApproachDate ASC
        LIMIT 1;
        """,

    "Find the average miss distance of hazardous vs non-hazardous asteroids":
        """
        SELECT a.isPotentiallyHazardousAsteroid AS hazardous, AVG(ca.astronomicalMissDistance) AS avg_miss_distance
        FROM close_approach ca
        JOIN asteroids a ON ca.noReferenceId = a.id
        GROUP BY hazardous;
        """,
}

# Predefined Queries Page
if select == "Home":
    st.sidebar.title("Predefined Asteroid Queries")
    selected_query = st.sidebar.selectbox("Choose a query to execute:", list(queries.keys()))

    # Execute query
    if selected_query:
        query_string = queries[selected_query]
        st.subheader(f"Result: {selected_query}")
        df = pd.read_sql_query(query_string, conn)
        st.dataframe(df)


Writing app.py


In [ ]:
!wget -q -O - ipv4.icanhazip.com   # this command will generate a password for you(copy that)

35.243.197.92


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501  # replace the file name accordingly



⠙⠹⠸⠼⠴⠦⠧
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.243.197.92:8501

⠇⠏⠋⠙⠹⠸⠼Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏your url is: https://floppy-tips-switch.loca.lt
  Stopping...
^C


Overwriting app.py


Cloning into 'asteroid-project'...
fatal: could not read Username for 'https://github.com': No such device or address
